In [1]:
# Dependencies
import json
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
# Import and read in CSV
youtube_df = pd.read_csv('./Data/USvideos.csv')

# Import Json and create category csv to read in excel if needed
# cat_id_df = pd.read_json('./Data/US_category_id.json')
# cat_id_df.to_csv('./Data/Category_alt.csv')

In [2]:
#  Check Dtypes
youtube_df.dtypes

video_id                  object
trending_date             object
title                     object
channel_title             object
category_id                int64
publish_time              object
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
comments_disabled           bool
ratings_disabled            bool
video_error_or_removed      bool
description               object
dtype: object

In [3]:
# Fix time columns
youtube_df["publish_time"] = pd.to_datetime(youtube_df["publish_time"], infer_datetime_format=True)
youtube_df['publish_time']= youtube_df['publish_time'].values.astype('datetime64[s]')
youtube_df["trending_date"] = pd.to_datetime(youtube_df["trending_date"], format="%y.%d.%m", exact=False)

# Create a column that shows how many days after a video is posted that it becomes trending
youtube_df['post_to_trend_days'] = (youtube_df['trending_date'] - youtube_df['publish_time']).dt.days

# Create a days trending column
youtube_df['days_trending']  = youtube_df.video_id.map(youtube_df.video_id.value_counts())

# Add a tags count

youtube_df = youtube_df.assign(tags_count = lambda x: x.tags.apply(lambda y: y.count('|')+1))

In [4]:
# Check for new columns and fixed data types
# youtube_df.dtypes

In [5]:
#  Find out what category id's are being used
# youtube_df.category_id.value_counts()

In [6]:
# Create a dictionary for the categories used
# Reference the 'Category_alt.csv'
# Create a dict
categories = {1:'Film & Animation',
              2:'Autos & Vehicles',
              10:'Music',
              15:'Pets & Animals',
              17:'Sports',
              19:'Travel & Events',
              20:'Gaming',
              22:'People & Blogs',
              23:'Comedy',
              24:'Entertainment',
              25:'News & Politics',
              26:'Howto & Style',
              27:'Education',
              28:'Science & Technology',
              29:'Nonprofits & Activism',
              43:'Shows' 
             }

In [7]:
# Create a duplicate of 'category_id' to apply the dictionary to
youtube_df['category_type'] = youtube_df.loc[:,'category_id']

# Apply the dictionary
youtube_df.category_type = [categories[item] for item in youtube_df.category_type]

# Drop columns that won't be used 
youtube_df = youtube_df.drop(['thumbnail_link'], axis=1)

In [8]:
# Check to see if the dictornary was applied properly
# youtube_df.category_type.value_counts()

In [9]:
# Get count of unique titles
# youtube_df.nunique()

In [10]:
# Turn video_id into string and create dataframe for the First day the video was trending 
youtube_df_f = youtube_df.copy()
youtube_df_f['video_id'] = youtube_df_f['video_id'].astype('string')

# Turn video_id into string and create a dataframe for the last day the video was trending.
youtube_df_l = youtube_df.copy()
youtube_df_l['video_id'] = youtube_df_l['video_id'].astype('string')

In [11]:
# Keep the last time a video id occured
youtube_df_f = youtube_df_f.drop_duplicates(subset=['video_id'],keep='first')
# Keep the last time a video id occured
youtube_df_l = youtube_df_l.drop_duplicates(subset=['video_id'],keep='last')

In [12]:
# youtube_df_f['days_trending'] = youtube_df.groupby('video_id').transform('count').fillna(0)

In [13]:
youtube_df_l.dtypes

video_id                          string
trending_date             datetime64[ns]
title                             object
channel_title                     object
category_id                        int64
publish_time              datetime64[ns]
tags                              object
views                              int64
likes                              int64
dislikes                           int64
comment_count                      int64
comments_disabled                   bool
ratings_disabled                    bool
video_error_or_removed              bool
description                       object
post_to_trend_days                 int64
days_trending                      int64
tags_count                         int64
category_type                     object
dtype: object

In [14]:
youtube_df_l.nunique()

video_id                  6351
trending_date              205
title                     6343
channel_title             2198
category_id                 16
publish_time              6267
tags                      5855
views                     6335
likes                     5633
dislikes                  2624
comment_count             3771
comments_disabled            2
ratings_disabled             2
video_error_or_removed       2
description               6201
post_to_trend_days         130
days_trending               30
tags_count                  65
category_type               16
dtype: int64

In [15]:
new_df = youtube_df_f[['video_id', 'trending_date', 'title', 'channel_title', 'category_id', 'publish_time', 'views', 'likes', 'dislikes', 'comment_count', 'post_to_trend_days', 'days_trending', 'tags_count', 'category_type' ]]

In [16]:
new_df

,video_id,trending_date,title,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,post_to_trend_days,days_trending,tags_count,category_type
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01,748374,57527,2966,15954,0,7,1,People & Blogs
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00,2418783,97185,6146,12703,0,7,4,Entertainment
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24,3191434,146033,5339,8181,1,7,23,Comedy
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04,343168,10172,666,2146,0,7,27,Entertainment
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41,2095731,132235,1989,17518,1,6,14,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40760,dS5Thrl-4Kc,2018-06-14,CRAYOLA MAKEUP | HIT OR MISS?,Laura Lee,26,2018-06-12 18:55:26,607422,26166,895,3517,1,1,21,Howto & Style
40761,JGm9Y_hFqNk,2018-06-14,First Take reacts: Kyrie Irving says contract ...,ESPN,17,2018-06-13 15:06:08,812832,7701,320,2505,0,1,31,Sports
40762,6h8QgZF5Qu4,2018-06-14,Drop the Mic w/ Ashton Kutcher & Sean Diddy Combs,The Late Late Show with James Corden,24,2018-06-13 05:27:27,864189,18811,399,1074,0,1,28,Entertainment
40764,mpnshdmtE2Y,2018-06-14,Carla Makes BA Smashburgers | From the Test Ki...,Bon Appétit,26,2018-06-12 16:03:58,540149,14206,693,1211,1,1,26,Howto & Style


In [17]:
# youtube_df_l.to_csv('./Data/Youtube_last_trending.csv')
# youtube_df_f.to_csv('./Data/Youtube_first_trending.csv')
new_df.to_csv('./Data/youtube_final_clean.csv')

In [18]:
new_df.dtypes

video_id                      string
trending_date         datetime64[ns]
title                         object
channel_title                 object
category_id                    int64
publish_time          datetime64[ns]
views                          int64
likes                          int64
dislikes                       int64
comment_count                  int64
post_to_trend_days             int64
days_trending                  int64
tags_count                     int64
category_type                 object
dtype: object